In [1]:
from operator import itemgetter
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
# import pymssql
# from sqlalchemy import create_engine
import os
from os import environ
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Load credentials
os.environ["OPENAI_API_KEY"] = os.getenv('AZURE_OPENAI_KEY')
os.environ["OPENAI_API_ENDPOINT"] = os.getenv('AZURE_OPENAI_ENDPOINT')
os.environ["OPENAI_API_VERSION"] = os.getenv('AZURE_OPENAI_API_VERSION')
os.environ["AZURE_OPENAI_DEPLOYMENT"] = os.getenv('AZURE_OPENAI_DEPLOYMENT')

In [3]:
import requests

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    # Open a local file in binary write mode
    with open("Chinook.db", "wb") as file:
        # Write the content of the response (the file) to the local file
        file.write(response.content)
    print("File downloaded and saved as Chinook.db")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as Chinook.db


In [4]:
# Load Azure OpenAI model
llm = AzureChatOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0
)

In [5]:
# Connect to db
db_uri = "sqlite:///Chinook.db"
db = SQLDatabase.from_uri(db_uri)

In [6]:
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [8]:
prefix = f"""Anda adalah asisten AI yang ahli dalam melakukan query pada Database Chinook.
Database ini berisi informasi tentang toko media digital, termasuk pelanggan, faktur, dan trek musik.
Jika sebuah informasi tidak ditemukan, coba merujuk pada aturan berikut:
- Saat melakukan join dengan tabel Customer, gunakan kolom CustomerId.
- Saat menanyakan tentang faktur, hubungkan dengan kolom BillingCountry dan Total.
Jika sebuah kata kunci tidak ditemukan, gunakan kamus berikut sebagai referensi:
- judul track = Name (dari Track)
- album = Title (dari Album)
- artis = Name (dari Artist)
"""
suffix = f"""Seorang pengguna ingin mengetahui informasi dari database Chinook.
"""

In [9]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm=llm,
                                db=db,
                                agent_type="openai-tools",
                                prefix=prefix,
                                suffix=suffix,
                                agent_executor_kwargs={"return_intermediate_steps": True},
                                verbose=True,
                                )

In [10]:
text_input = "album terlaris sepanjang masa."

In [11]:
response = agent_executor.invoke({
"input": text_input
})
queries = []
for (log, output) in response["intermediate_steps"]:
    if log.tool == 'sql_db_query':
        queries.append(log.tool_input)




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Album'}`



CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/
Invoking: `sql_db_schema` with `{'table_names': 'InvoiceLine'}`



CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("InvoiceLineId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("InvoiceId") REFERENCES "Invoic

In [12]:
print(response['output'])

Album terlaris sepanjang masa di database Chinook adalah **"Minha Historia"** dengan total penjualan sebanyak 27 kali.


In [14]:
if queries:
    print("The Query is...")
    print(queries[0]['query'])

The Query is...
SELECT a.Title AS Album, COUNT(il.InvoiceLineId) AS Sales
FROM Album a
JOIN Track t ON a.AlbumId = t.AlbumId
JOIN InvoiceLine il ON t.TrackId = il.TrackId
GROUP BY a.Title
ORDER BY Sales DESC
LIMIT 1;
